<a href="https://colab.research.google.com/github/pujangrg1/pujan_INFO5731_Spring2020/blob/main/In_class_exercise_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The seventh in-class-exercise (20 points in total, 3/16/2021)**

Question description: In the last in-class-exercise (exercise-06), you collected the titles of 100 articles about data science, natural language processing, and machine learning. The 100 article titles will be used as the text corpus of this exercise. Perform the following tasks:

## (1) (8 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [31]:
# Write your code here
import csv
import pandas as pd
file = []
articleTitles = pd.read_csv('articleList.csv', encoding = "ISO-8859-1")
articleTitles

,Unnamed: 0,Title
0,0,Data science in action
1,1,Data science and prediction
2,2,Data science and its relationship to big data ...
3,3,[BOOK][B] Data Science for Business: What you ...
4,4,[BOOK][B] High-dimensional probability: An int...
...,...,...
95,95,Computational optimal transport: With applicat...
96,96,"Data science, predictive analytics, and big da..."
97,97,"Reproducible, interactive, scalable and extens..."
98,98,The quantified self: Fundamental disruption in...


In [32]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
!pip install pyLDAvis

In [34]:
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version
install_java()

openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [35]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (3.8.3)


In [36]:
import spacy.cli
spacy.cli.download("en")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [37]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [38]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [39]:
article_titles = pd.read_csv('/content/articleList.csv', encoding = "ISO-8859-1")
article_titles.head(10)

,Unnamed: 0,Title
0,0,Data science in action
1,1,Data science and prediction
2,2,Data science and its relationship to big data ...
3,3,[BOOK][B] Data Science for Business: What you ...
4,4,[BOOK][B] High-dimensional probability: An int...
5,5,Computational optimal transport: With applicat...
6,6,"Data science, predictive analytics, and big da..."
7,7,"Reproducible, interactive, scalable and extens..."
8,8,The quantified self: Fundamental disruption in...
9,9,[HTML][HTML] Big data: astronomical or genomical?


In [40]:
data = article_titles['Title'].tolist()

In [42]:
#Tokenizing words and cleaning up text using Gensim's simple_preprocess()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words)

[['data', 'science', 'in', 'action'], ['data', 'science', 'and', 'prediction'], ['data', 'science', 'and', 'its', 'relationship', 'to', 'big', 'data', 'and', 'data', 'driven', 'decision', 'making'], ['book', 'data', 'science', 'for', 'business', 'what', 'you', 'need', 'to', 'know', 'about', 'data', 'mining', 'and', 'data', 'analytic', 'thinking'], ['book', 'high', 'dimensional', 'probability', 'an', 'introduction', 'with', 'applications', 'in', 'data', 'science'], ['computational', 'optimal', 'transport', 'with', 'applications', 'to', 'data', 'science'], ['data', 'science', 'predictive', 'analytics', 'and', 'big', 'data', 'revolution', 'that', 'will', 'transform', 'supply', 'chain', 'design', 'and', 'management'], ['reproducible', 'interactive', 'scalable', 'and', 'extensible', 'microbiome', 'data', 'science', 'using', 'qiime'], ['the', 'quantified', 'self', 'fundamental', 'disruption', 'in', 'big', 'data', 'science', 'and', 'biological', 'discovery'], ['html', 'html', 'big', 'data', '

In [43]:
#Creating Bigram and Trigram Models
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [44]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [45]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

[['datum', 'science', 'action'], ['datum', 'science', 'prediction'], ['datum', 'science', 'relationship', 'big', 'datum', 'datum', 'drive', 'decision', 'making'], ['science', 'business', 'nee', 'know', 'datum', 'mining', 'datum', 'analytic', 'thinking'], ['high', 'dimensional', 'probability', 'introduction', 'application', 'datum', 'science'], ['transport', 'application', 'datum', 'science'], ['predictive', 'analytic', 'datum', 'revolution', 'transform', 'supply', 'chain', 'design', 'management'], ['scalable', 'extensible', 'datum', 'science', 'use'], ['quantified', 'self', 'fundamental', 'science', 'biological', 'discovery'], [], ['datum', 'science', 'action'], ['datum', 'science', 'prediction'], ['datum', 'science', 'relationship', 'big', 'datum', 'datum', 'drive', 'decision', 'making'], ['science', 'business', 'nee', 'know', 'datum', 'mining', 'datum', 'analytic', 'thinking'], ['high', 'dimensional', 'probability', 'introduction', 'application', 'datum', 'science'], ['transport', 'a

In [46]:
#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [47]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('action', 1), ('datum', 1), ('science', 1)]]

In [57]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [58]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.089*"datum" + 0.067*"science" + 0.031*"nee" + 0.031*"mining" + '
  '0.031*"transport" + 0.031*"application" + 0.031*"thinking" + '
  '0.031*"analytic" + 0.031*"business" + 0.031*"prediction"'),
 (1,
  '0.218*"datum" + 0.109*"analytic" + 0.109*"thinking" + 0.109*"know" + '
  '0.109*"business" + 0.109*"mining" + 0.109*"science" + 0.109*"nee" + '
  '0.001*"supply" + 0.001*"transform"'),
 (2,
  '0.052*"datum" + 0.039*"science" + 0.030*"mining" + 0.030*"drive" + '
  '0.030*"know" + 0.030*"relationship" + 0.030*"making" + 0.030*"decision" + '
  '0.030*"nee" + 0.030*"thinking"'),
 (3,
  '0.067*"datum" + 0.047*"science" + 0.034*"relationship" + 0.034*"decision" + '
  '0.034*"drive" + 0.034*"big" + 0.034*"making" + 0.032*"transport" + '
  '0.032*"application" + 0.028*"action"'),
 (4,
  '0.162*"science" + 0.162*"self" + 0.162*"biological" + 0.162*"quantified" + '
  '0.162*"fundamental" + 0.162*"discovery" + 0.001*"datum" + 0.001*"analytic" '
  '+ 0.001*"revolution" + 0.001*"predictive"

## (2) (8 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
#import modules
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

**Create an LSA model using Gensim**

In [ ]:
# Write your code here
lsamodel = LsiModel(corpus, 20, id2word)

**Determine the number of topics**

In [ ]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=20, id2word = dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

**Coherence score values**

In [ ]:
start,stop,step=2,60,6
model_list_lsa, coherence_values_lsa = compute_coherence_values(id2word, corpus,data_lemmatized,stop, start, step)

In [ ]:
for no_of_topics, cv in zip(range(2, 60, 6), coherence_values_lsa):
  print("Num Topics:", no_of_topics, " - Coherence Value:", round(cv, 4))

Num Topics: 2  - Coherence Value: 0.4226
Num Topics: 8  - Coherence Value: 0.4122
Num Topics: 14  - Coherence Value: 0.4348
Num Topics: 20  - Coherence Value: 0.4817
Num Topics: 26  - Coherence Value: 0.4297
Num Topics: 32  - Coherence Value: 0.4469
Num Topics: 38  - Coherence Value: 0.4451
Num Topics: 44  - Coherence Value: 0.423
Num Topics: 50  - Coherence Value: 0.4523
Num Topics: 56  - Coherence Value: 0.4326


**Print Topics**

In [ ]:
pprint(lsamodel.print_topics())

[(0,
  '0.792*"datum" + 0.453*"science" + 0.128*"analytic" + 0.115*"making" + '
  '0.115*"relationship" + 0.115*"decision" + 0.115*"drive" + 0.115*"big" + '
  '0.099*"application" + 0.088*"thinking"'),
 (1,
  '0.357*"analytic" + -0.319*"science" + 0.303*"transform" + 0.303*"design" + '
  '0.303*"management" + 0.303*"predictive" + 0.303*"revolution" + '
  '0.303*"chain" + 0.303*"supply" + -0.139*"application"'),
 (2,
  '0.347*"science" + 0.255*"discovery" + 0.255*"self" + 0.255*"fundamental" + '
  '0.255*"quantified" + 0.255*"biological" + -0.232*"drive" + '
  '-0.232*"decision" + -0.232*"relationship" + -0.232*"making"'),
 (3,
  '-0.351*"mining" + -0.351*"know" + -0.351*"business" + -0.351*"thinking" + '
  '-0.351*"nee" + -0.218*"analytic" + 0.202*"application" + 0.151*"high" + '
  '0.151*"dimensional" + 0.151*"introduction"'),
 (4,
  '0.359*"application" + -0.291*"fundamental" + -0.291*"self" + '
  '-0.291*"discovery" + -0.291*"quantified" + -0.291*"biological" + '
  '0.281*"probabili

## (3) (4 points) Compare the results generated by the two topic modeling algorithms, which one is better? You should explain the reasons in details.

Each of the two modeling algorithms has had varying degrees of success. With LDA, i was able to generate 20 topics where as with LSA outputed only 10 topics. LSA generated a more coherent topic-set and a more reasonable topic distribution. LDA achieved nearly the opposite: separation between topics was very good, but the topics derived were not especially intelligible, and their distribution seemed unlikely.
But regardless, the above explorations establish a strong foundation for any further analyses of the dataset – and moreover, clearly demonstrate the viability of topic modelling on an ostensibly novel data format.